# Importing and some EDA of the r/WallStreetBets dataset

# Goals for today

- text cleaning
- count vectorizer
- run first nlp model for topic analysis
- look into spacy

## Roadmap for data exploration and processing

- parse down data to set i want to work with
    - see cell down below talking about what data i want to focus on
        - title
        - selftext
        - created_utc
    - export to a csv for easier pickup where i left off
       

- text processing stuff for nltk
    - actual preprocessing steps (use the functions from the NLP lecture and the NLP case study)
        - remove punctuation (should i remove newline chars here?)
        - remove stopwords
        - remove additional spaces, digits, (or newline characters here?)
        - lemmatize/stem
        - tfidf?
        - now ready to model
    - get some topics from the posts
    - also look into SpaCy for maybe a different analysis/point of view


- clustering model


- sentiment analysis model?

In [74]:
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [11]:
pd.set_option('display.max_columns', None)

import the data and some initial inspection of the dataframe

In [2]:
posts = pd.read_csv('data/wallstreetbets_posts.csv')

/Users/calebjones/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,81,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
posts.shape

(707461, 85)

In [4]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707461 entries, 0 to 707460
Data columns (total 85 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   all_awardings                  707461 non-null  object 
 1   allow_live_comments            707461 non-null  bool   
 2   author                         707461 non-null  object 
 3   author_flair_background_color  1121 non-null    object 
 4   author_flair_css_class         76 non-null      object 
 5   author_flair_richtext          649545 non-null  object 
 6   author_flair_text              22093 non-null   object 
 7   author_flair_text_color        80023 non-null   object 
 8   author_flair_type              649545 non-null  object 
 9   author_fullname                649545 non-null  object 
 10  author_patreon_flair           649545 non-null  object 
 11  author_premium                 649545 non-null  object 
 12  awarders                      

In [15]:
posts.head()

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_template_id,link_flair_text,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,pwls,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,title,total_awards_received,treatment_tags,upvote_ratio,url,whitelist_status,wls,removed_by_category,post_hint,preview,thumbnail_height,thumbnail_width,url_overridden_by_dest,media_metadata,gallery_data,is_gallery,media,media_embed,secure_media,secure_media_embed,banned_by,author_flair_template_id,author_cakeday,edited,gilded,distinguished,collections,crosspost_parent,crosspost_parent_list
0,[],False,readyrummy1,NaN,NaN,"[{'e': 'text', 't': '\x1b201202:4:1'}]",201202:4:1,dark,richtext,t2_7ppu74oe,False,False,[],False,False,1607957430,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{},kcz0hf,True,False,False,False,True,True,False,#9e8d49,storytime,"[{'e': 'text', 't': 'Storytime'}]",f86d7f4a-4e9b-11e9-a6f9-0e03190c749e,Storytime,light,richtext,False,False,False,6,0,False,no_ads,/r/wallstreetbets/comments/kcz0hf/ruled_by_the...,False,0.0,1607957441,1,5:30 AM\n\nWoke up five minutes ago. Made som...,True,False,False,wallstreetbets,t5_2th52,1697978,public,confidence,self,Ruled by the Theta Gods,0,[],1.0,https://www.reddit.com/r/wallstreetbets/commen...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Pluto_Muto,NaN,NaN,[],NaN,NaN,text,t2_9bo9tvhh,False,False,[],False,False,1607957404,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{},kcz07r,False,False,False,False,False,True,False,#0392cf,yolo,"[{'e': 'text', 't': 'YOLO'}]",da18a43a-83c5-11e8-9b6c-0e287561ddb8,YOLO,light,richtext,False,False,True,0,0,False,NaN,/r/wallstreetbets/comments/kcz07r/3k_and_i_wan...,False,NaN,1607957417,1,[removed],True,False,False,wallstreetbets,t5_2th52,1697977,public,confidence,self,3k and I want more,0,[],1.0,https://www.reddit.com/r/wallstreetbets/commen...,NaN,NaN,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Hiro08,NaN,NaN,[],NaN,NaN,text,t2_17panrff,False,False,[],False,False,1607957384,i.redd.it,https://www.reddit.com/r/wallstreetbets/commen...,{},kcyzzz,False,False,False,True,False,False,False,#349e48,profit,"[{'e': 'text', 't': 'Gain'}]",6bd9394c-0a95-11e9-82db-0eb3a11267a0,Gain,light,richtext,False,False,True,0,0,False,no_ads,/r/wallstreetbets/comments/kcyzzz/thank_you_fe...,False,0.0,1607957395,1,NaN,True,False,False,wallstreetbets,t5_2th52,1697978,public,confidence,https://b.thumbs.redditmedia.com/pl2VfNx7n9Iad...,Thank you fellow retards and autists! Big ston...,0,[],1.0,https://i.redd.it/i6ae8w4c16561.jpg,no_ads,0.0,moderator,image,"{'enabled': True, 'images': [{'id': 'j_Hi_h_tu...",140.0,140.0,https://i.redd.it/i6ae8w4c16561.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,uslashuname,NaN,NaN,[],NaN,NaN,text,t2_3nee5zpr,False,False,[],False,False,1607957282,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,{},kcyyw4,False,False,False,False,False,True,False,#800080,question,"[{'e': 'text', 't': 'Discussion'}]",96f6c79e-b853-11e5-a4cb-0ebdf030e05d,Discussion,light,richtext,False,False,True,0,0,False,no_ads,/r/wallstreetbets/comments/kcyyw4/azn_fair_val...,False,0.0,1607957293,1,[removed],True,False,False,wallstreetbets,t5_2th52,1697974,

In [78]:
# quite a lot of the selftext column are NaN or [removed], this is a look at the possible reasons why they
# are removed 
pd.unique(posts.removed_by_category)

array([nan, 'moderator', 'deleted', 'automod_filtered', 'reddit',
       'anti_evil_ops', 'author'], dtype=object)

In [13]:
posts.selftext[0]

'5:30 AM\n\nWoke up five minutes ago.  Made some instant coffee because drip would take too long.  \n\nWatching the e-mini tick because the 1m is too slow.\n\nHeard in the background my girlfriend muttering something along the lines of "grow up" as she takes her suitcase and slam the door, but it barely registers.  12/16 SPYc is all I can think of at the moment.\n\nNormally I wouldn\'t be shaking, but after losing 50% of my account in the past week I decided to go all-in to get back to even.  Once I am back in the black things will be different.  I won\'t do this again, I swear.  It is just a one time thing.  I promise the Gods that if I can expire ITM I will stick to strict bankroll management.  Vertical spreads only, maximum 5% per trade.\n\nCan\'t think about losing on this trade because it would be too painful.  My brain is locked into pure optimism.  No plans what I will do if I lose so why worry?\n\nEach downward tick brings a shock of pain to my entire body.  Every sharp uptick 

looking through the columns contained in the dataset, info that i am mainly interested in is:

- selftext - this is the post/comment text
    - remove newline data
    - expand contractions
    - **there are 707461 posts and only 424107 have text still available at the time of scraping**
        - need to filter out [removed] and NaN
    - other thoughts about text processing here...
- title - could do a sentiment analysis on the titles?
    - all the posts' titles are available even if the post has been removed or deleted, so this is a good approach
    - after looking at the titles in the first 5 posts, definitely want to do this...
        - 'thank you fellow retards and autists' and 'i wanna be less retarded' geez hahaah
- created_utc - this is the time stamp of the post creation data in UTC, recorded in unix time code

- what is the text of the common emojis? **they are actually emojis, not unicode in this dataset**

In [80]:
# columns to include in pared down dataframe
cols = ['author', 'title', 'selftext', 'created_utc', 'num_comments', 'subreddit_subscribers', 'upvote_ratio']

# select a subset of the full posts data to work with
main_posts = posts[cols].copy()
main_posts.dropna(axis=0, inplace=True)
main_posts = main_posts[main_posts.selftext != '[removed]']
main_posts

,author,title,selftext,created_utc,num_comments,subreddit_subscribers,upvote_ratio
0,readyrummy1,Ruled by the Theta Gods,5:30 AM\n\nWoke up five minutes ago. Made som...,1607957430,6,1697978,1.0
5,officespace2020,Motivation Monday,"# Intro\n\nLet me get soft with ya, it can get...",1607957260,9,1697974,1.0
9,BigSlimetime73,Toy Yoda (but the car company),"Hello fellow retards,\n\nThis weekend I came a...",1607957127,4,1697963,1.0
12,shroomsAndWrstershir,"I've never been so happy to lose $300,000 in m...","After Saturday's SPCE failed launch, I though ...",1607957052,85,1697960,1.0
26,alphabluepiller,Nasdaq-100 changes,"NEW YORK, Dec. 11, 2020 (GLOBE NEWSWIRE) -- Na...",1607955881,7,1697898,1.0
...,...,...,...,...,...,...,...
707450,Dirtball26,WHY THE FUCK WONT ROBINHOOD LET ME OPTIONS TRADE.,Haven’t used since I got my free stock last ye...,1609094822,6,1746192,1.0
707451,Salahforisis,You fucking knuckleheads got my mom to reinves...,I don't know how the fuck you guys became so p...,1609094763,337,1746190,1.0
707454,Junkbot,GME 🚀🚀🚀🚀🚀🚀 = gold?,Saw the trends and trying a back test to see i...,1609093826,4,1746160,1.0
707459,ZookeepergameSoggy49,Was it a dream??!😴 😴 GME opens on Monday up 100%,Large GameStop Investor Ryan Cohen Buys More S...,1609093189,18,1746143,1.0


In [85]:
# from posts dataframe (original full posts)
min_date = min(posts.created_utc)
max_date = max(posts.created_utc)
print('earliest date was ' + str(dt.fromtimestamp(min_date)))
print('latest date was ' + str(dt.fromtimestamp(max_date)))

earliest date was 2020-12-08 07:51:38
latest date was 2021-02-03 22:42:21


In [86]:
# from main_posts dataframe, the pared down one i selected columns of interest from
min_date = min(main_posts.created_utc)
max_date = max(main_posts.created_utc)
print('earliest date was ' + str(dt.fromtimestamp(min_date)))
print('latest date was ' + str(dt.fromtimestamp(max_date)))

earliest date was 2020-12-08 07:51:38
latest date was 2021-02-03 22:42:20


In [87]:
# export to csv, index = False is to prevent it from making a dumb extra index column. don't need that here.
main_posts.to_csv('data/main_posts.csv', index = False)

In [88]:
# test read back in

main = pd.read_csv('data/main_posts.csv')
main.head()

,author,title,selftext,created_utc,num_comments,subreddit_subscribers,upvote_ratio
0,readyrummy1,Ruled by the Theta Gods,5:30 AM\n\nWoke up five minutes ago. Made som...,1607957430,6,1697978,1.0
1,officespace2020,Motivation Monday,"# Intro\n\nLet me get soft with ya, it can get...",1607957260,9,1697974,1.0
2,BigSlimetime73,Toy Yoda (but the car company),"Hello fellow retards,\n\nThis weekend I came a...",1607957127,4,1697963,1.0
3,shroomsAndWrstershir,"I've never been so happy to lose $300,000 in m...","After Saturday's SPCE failed launch, I though ...",1607957052,85,1697960,1.0
4,alphabluepiller,Nasdaq-100 changes,"NEW YORK, Dec. 11, 2020 (GLOBE NEWSWIRE) -- Na...",1607955881,7,1697898,1.0


In [89]:
main_posts.shape

(96482, 7)

In [90]:
# working on getting rid of [removed] and NaN from the selftext column while also converting to just a 
# pandas series type for looking into emoji characters

# using the next line to convert a dataframe column into a series with the .squeeze() method
# my_series = df['Last_Name'].squeeze()

selftext = posts[posts.selftext != '[removed]'].dropna(axis = 0, subset=['selftext']).selftext.squeeze()

In [91]:
selftext

0         5:30 AM\n\nWoke up five minutes ago.  Made som...
5         # Intro\n\nLet me get soft with ya, it can get...
9         Hello fellow retards,\n\nThis weekend I came a...
12        After Saturday's SPCE failed launch, I though ...
26        NEW YORK, Dec. 11, 2020 (GLOBE NEWSWIRE) -- Na...
                                ...                        
707450    Haven’t used since I got my free stock last ye...
707451    I don't know how the fuck you guys became so p...
707454    Saw the trends and trying a back test to see i...
707459    Large GameStop Investor Ryan Cohen Buys More S...
707460      \n\n# Trading Plan 2021\n\n# Daily Routine (...
Name: selftext, Length: 96482, dtype: object

In [ ]:
🚀

In [92]:
# now that i have selftext as a series, i can use .str.contains to search for specific text, emojis in this case
# turns out the emojis are actually emojis, not unicode translations of them (U+ stuff)
# can NLP and clustering handle emojis

emojis = selftext.str.contains('🚀', regex=False)
emojis

0         False
5         False
9         False
12        False
26        False
          ...  
707450    False
707451    False
707454     True
707459    False
707460    False
Name: selftext, Length: 96482, dtype: bool

In [56]:
pd.unique(emojis)

array([False,  True])

In [96]:
test1 = posts.selftext[707454].lower()
test1

'saw the trends and trying a back test to see if hypothesis is true. \n\nalso, gme 🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀'

In [97]:
test2 = posts.title[707459].lower()
test2

'was it a dream??!😴 😴 gme opens on monday up 100%'

In [37]:
# dataframe of posts by DeepFuckingValue, the godfather of WSB
dfv = posts[posts['author'] == 'DeepFuckingValue']

In [39]:
dfv.shape

(18, 85)